In [4]:
import fsspec
import os
from ipfsspec.asyn import AsyncIPFSFileSystem
from fsspec import register_implementation
import asyncio
import io

# register_implementation(IPFSFileSystem.protocol, IPFSFileSystem)
register_implementation(AsyncIPFSFileSystem.protocol, AsyncIPFSFileSystem)

# with fsspec.open("ipfs://QmZ4tDuvesekSs4qM5ZBKpXiZGun7S2CYtEZRB3DYXkjGx", "r") as f:
#     print(f.read())
class fs:
    ipfs = fsspec.filesystem("ipfs")
    file = fsspec.filesystem("file")

# Changing Gateways

In [5]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


In [6]:
fs.ipfs.gateway.gws

[(<ipfsspec.gateway.GatewayState at 0x7fbe55412d60>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fbe55412d00>),
 (<ipfsspec.gateway.GatewayState at 0x7fbe55412c40>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fbe55412ac0>),
 (<ipfsspec.gateway.GatewayState at 0x7fbe55412fa0>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fbe55412e50>),
 (<ipfsspec.gateway.GatewayState at 0x7fbe55412ee0>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7fbe55412df0>)]

In [7]:
fs.ipfs.gateway.gws[0][1].url

'https://ipfs.io'

In [8]:
fs.ipfs.change_gateway_type = 'local'

Changed to local node


In [9]:
fs.ipfs.gateway.gws[0][1].url

'http://127.0.0.1:8080'

# Local

## Put

#### Put File

In [10]:
# put in a file and store it into test folder 'fam'
put_file_res = fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_put_file')
fs.ipfs.ls('/test_put_file')

[{'name': '/test_put_file/yo.txt',
  'CID': 'QmNuMqLrDoSEZeZxyPWUjFZv9HVYwmHB9n5PEwpfoJyAD6',
  'type': 'file',
  'size': 13}]

#### Put Folder

In [11]:
# put a directory and store it into test folder
fs.ipfs.put(path='test/data/input', rpath='/test_put_folder' ,recursive=True, return_cid=False)
fs.ipfs.ls('/')

[{'name': '/test_get_file',
  'CID': 'QmQFuaEFT1pmyDmh8bx5peApmXXoA5D3CxZ1HfxBztt1cj',
  'type': 'directory',
  'size': 0},
 {'name': '/test_get_folder',
  'CID': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7',
  'type': 'directory',
  'size': 0},
 {'name': '/test_put_file',
  'CID': 'QmQFuaEFT1pmyDmh8bx5peApmXXoA5D3CxZ1HfxBztt1cj',
  'type': 'directory',
  'size': 0},
 {'name': '/test_put_folder',
  'CID': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7',
  'type': 'directory',
  'size': 0},
 {'name': '/test_rm_file',
  'CID': 'QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn',
  'type': 'directory',
  'size': 0}]

## Cat

#### Cat File

In [12]:
# Given a file CID to cat
fs.ipfs.cat('QmNuMqLrDoSEZeZxyPWUjFZv9HVYwmHB9n5PEwpfoJyAD6')

b'yofvervevdfvd'

In [13]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_file/yo.txt')

b'yofvervevdfvd'

#### Cat Folder

In [14]:
# Given a directory CID to cat
fs.ipfs.cat('Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7')

{'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/bruh.txt': b'bruuhhhhhhhhh',
 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/yo.txt': b'yofvervevdfvd'}

In [15]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_folder')

{'/test_put_folder/bruh.txt': b'bruuhhhhhhhhh',
 '/test_put_folder/yo.txt': b'yofvervevdfvd'}

## Remove

#### Remove File

In [16]:
fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_rm_file' ,recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('test_rm_file', detail=False))
fs.ipfs.rm('/test_rm_file/yo.txt')
print('After: ',fs.ipfs.ls('test_rm_file', detail=False))

Before:  ['/test_rm_file/yo.txt']
After:  []


#### Remove Folder

In [17]:
fs.ipfs.put(path='test/data', rpath='/test_rm_folder', recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('test_rm_folder', detail=False))
fs.ipfs.rm('/test_rm_folder')
print('After: ',fs.ipfs.ls('test_rm_folder', detail=False))

Before:  ['/test_rm_folder/input', '/test_rm_folder/output']
After:  []


## Get

#### Get Remote File - MFS Path

In [18]:
if fs.file.exists('test/data/yo.txt'):
    fs.file.rm('test/data/yo.txt')
    
fs.ipfs.put(lpath='test/data/input/yo.txt', rpath='/test_get_file')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])
fs.ipfs.get(rpath='/test_get_file/yo.txt', lpath='test/data',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

Before:  []
After:  ['data/yo.txt']


#### Get Remote File - CID Path 

*not working*

In [19]:
if fs.file.exists('test/data/yo.txt'):
    fs.file.rm('test/data/yo.txt')
    
fs.ipfs.put(lpath='test/data/input/yo.txt', rpath='/test_get_file')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

fs.ipfs.get(rpath='QmNuMqLrDoSEZeZxyPWUjFZv9HVYwmHB9n5PEwpfoJyAD6', lpath='test/data',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

Before:  []
After:  []


#### Get Remote Folder - MFS Path

In [20]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])
fs.ipfs.get(rpath='/test_get_folder', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Before:  []
After:  ['data/output/bruh.txt', 'data/output/yo.txt']


#### Get Remote Folder - CID Path

In [21]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

fs.ipfs.get(rpath='Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Before:  []
After:  ['data/output/bruh.txt', 'data/output/yo.txt']


# Public

In [22]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


## Cat

#### Cat File

In [3]:
# Given a file CID to cat
r = fs.ipfs.cat('QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9')

ContentTypeError: 0, message='Attempt to decode JSON with unexpected mimetype: text/plain; charset=utf-8', url=URL('https://ipfs.io/api/v0/files/stat?arg=QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9')

#### Cat Folder

In [ ]:
# Given a directory CID to cat
fs.ipfs.cat('Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7')

## Get

#### Get Remote File - CID Path 


#### Get Remote Folder - CID Path